# PCA5

## 전처리

아래는 지정한 폴더의 엑셀파일들을 불러오는 전처리 코드입니다.

In [ ]:
import pandas as pd
import os

# 현재 스크립트의 경로 가져오기
current_folder = os.getcwd()

folder_path = os.path.join(current_folder, 'train')
file_names = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.xlsx') and (not file_name.startswith("~$"))]
print(file_names)

data_points = []  # 섹션 구분 없이 모든 데이터를 저장할 리스트

def process_files_in_folder():
    global data_points

    section_files = [file_name for file_name in file_names if not file_name.startswith("~$")]
    for file_name in section_files:
        print(file_name)
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_excel(file_path, header=0, engine='openpyxl')

            value_xx = df["xx"].tolist() if "xx" in df.columns else df[0].tolist()
            value_xy = df["xy"].tolist() if "xy" in df.columns else df[1].tolist()
            value_xz = df["xz"].tolist() if "xz" in df.columns else df[2].tolist()
            value_yx = df["yx"].tolist() if "yx" in df.columns else df[3].tolist()
            value_yy = df["yy"].tolist() if "yy" in df.columns else df[4].tolist()
            value_yz = df["yz"].tolist() if "yz" in df.columns else df[5].tolist()
            value_zx = df["zx"].tolist() if "zx" in df.columns else df[6].tolist()
            value_zy = df["zy"].tolist() if "zy" in df.columns else df[7].tolist()
            value_zz = df["zz"].tolist() if "zz" in df.columns else df[8].tolist()

            for xx, xy, xz, yx, yy, yz, zx, zy, zz in zip(value_xx, value_xy, value_xz,
                                                           value_yx, value_yy, value_yz,
                                                           value_zx, value_zy, value_zz):
                data_points.append((xx, xy, xz, yx, yy, yz, zx, zy, zz))

        except Exception as e:
            print(f"\n\n\nError reading file '{file_name}': {e}\n\n\n")

process_files_in_folder()

## PCA

pca차원축소를 진행하는 코드입니다.

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.decomposition import PCA

# 섹션 이름 리스트
sections = ["front", "left_side", "right_side", "left_up", "left_down", "right_up", "right_down", "unknown"]

# 데이터 처리
results = {}

# 데이터 포인트를 2D 배열로 변환
data_points = np.array(data_points)

# 데이터 포인트가 2D 배열임을 확인
if len(data_points.shape) == 1:
    data_points = data_points.reshape(-1, 9)  # 9차원 데이터

print("데이터 포인트 재구성 후 형태:", data_points.shape)

# PCA를 사용하여 데이터를 5차원으로 축소
pca = PCA(n_components=5)  # n_components를 5으로 변경
data_3d = pca.fit_transform(data_points)

print("설명된 분산 비율:", pca.explained_variance_ratio_)

In [ ]:
pca_df = pd.DataFrame(data_3d, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
pca_df

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

# DataFrame 생성
pca_df = pd.DataFrame(data_3d, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

# DataFrame을 XLSX 파일로 저장
pca_df.to_excel("pca_data.xlsx", index=False)

## 클러스터링 코드

아래는 xx xy xz yx yy yz zx zy zz의 9차원 데이터를 2차원데이터로 변환하여 k-means 클러스터링한 다음 로지스틱회귀를 통해 예측 모델을 만드는 코드입니다.

In [ ]:
# 클러스터와 섹션 매핑 실험
cluster_section_mapping = {0: 'right_down', 1: 'right_up', 2: 'front', 3: 'right_side', 4: 'unknown', 5: 'left_down', 6: 'left_side', 7: 'left_up' }

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# 클러스터링 결과를 일관성 있게 만들기 위한 랜덤 시드 고정
np.random.seed(42)

# K-means 클러스터링 수행
num_clusters = 8
kmeans = KMeans(n_clusters=num_clusters, init="k-means++")
kmeans.fit(data_3d)
labels = kmeans.labels_

print("클러스터 레이블:", labels)

# 클러스터 레이블에 대응하는 섹션 이름 리스트 생성
mapped_sections = [cluster_section_mapping[cluster_label] for cluster_label in labels]

# 모델 학습 및 평가
X_train, X_test, y_train, y_test = train_test_split(data_3d, mapped_sections, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("정확도:", accuracy)

# F1 점수 계산
f1 = f1_score(y_test, predictions, average='weighted')  # weighted F1 score를 계산

print("F1 점수:", f1)

In [ ]:
from collections import Counter

# 각 클러스터에 속한 데이터 포인트 수 계산
cluster_counts = Counter(labels)

# 클러스터 레이블을 오름차순으로 정렬
sorted_cluster_labels = sorted(cluster_counts.keys())

# 클러스터별 데이터 포인트 수 출력
for cluster_label in sorted_cluster_labels:
    count = cluster_counts[cluster_label]
    section = cluster_section_mapping.get(cluster_label, 'unknown')
    print(f"클러스터 {cluster_label} ({section}): {count}개 데이터 포인트")


In [ ]:
# 클러스터의 중심점 얻기
cluster_centers = kmeans.cluster_centers_

# 클러스터의 범위(반경) 설정 (예: 0.5 단위)
cluster_radius = 0.5

# 클러스터 범위 설정 및 출력
cluster_ranges = {}
for cluster_label, center in enumerate(cluster_centers):
    # 각 클러스터의 중심점을 기준으로 클러스터 범위 설정
    cluster_range = [center - cluster_radius, center + cluster_radius]
    cluster_ranges[cluster_label] = cluster_range

# 클러스터의 범위 출력
for cluster_label, cluster_range in cluster_ranges.items():
    section = cluster_section_mapping.get(cluster_label, 'unknown')
    print(f"클러스터 {section}의 범위:")
    print(f"최소 값: {cluster_range[0]}")
    print(f"최대 값: {cluster_range[1]}")
    print()

In [ ]:
import plotly.express as px

# 원래 데이터포인트와 PCA 축소된 데이터를 결합
pca_df = pd.DataFrame(data_3d, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

# 원래 데이터포인트에서 섹션 정보를 가져옴
pca_df['Section'] = mapped_sections

# 섹션 이름 변경
pca_df['Section'] = pca_df['Section'].replace({
    'front': 'IO (1)',
    'left_side': 'LMO (2)',
    'right_side': 'RMO (3)',
    'left_up': 'LUMC (4)',
    'left_down': 'LWMC (5)',
    'right_up': 'RUMC (6)',
    'right_down': 'RWMC (7)',
    'unknown': 'unknown'
})

# 데이터 시객화
fig = px.scatter_3d(pca_df, x='PC1', y='PC2', z='PC3', color='Section', title='PCA 3D 시각화',
                    category_orders={'Section': ['IO (1)', 'LMO (2)', 'RMO (3)', 'LUMC (4)', 'LWMC (5)', 'RUMC (6)', 'RWMC (7)', 'unknown']})

# 그래프의 크기와 해상도 조절
fig.update_layout(width=800, height=600)

# 글꼴 크기 조절
fig.update_layout(
    font=dict(
        size=20  # 글꼴 크기를 20으로 설정
    )
)

# x,y,z 축 숫자 레이블의 글꼴 크기 설정
fig.update_layout(scene=dict(
    xaxis=dict(
        title_font=dict(size=17),
        tickfont=dict(size=10),   # x 축 숫자 레이블의 글꼴 크기를 설정
    ),
    yaxis=dict(
        title_font=dict(size=17),
        tickfont=dict(size=10),   # y 축 숫자 레이블의 글꼴 크기를 설정
    ),
    zaxis=dict(
        title_font=dict(size=17),
        tickfont=dict(size=10),   # z 축 숫자 레이블의 글꼴 크기를 설정
    )  
))

fig.show()

In [ ]:
import plotly.express as px

# 원래 데이터포인트와 PCA 축소된 데이터를 결합
pca_df = pd.DataFrame(data_3d, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

# 원래 데이터포인트에서 섹션 정보를 가져옴
pca_df['Section'] = mapped_sections

# 섹션 이름 변경
pca_df['Section'] = pca_df['Section'].replace({
    'front': 'IO (1)',
    'left_side': 'LMO (2)',
    'right_side': 'RMO (3)',
    'left_up': 'LUMC (4)',
    'left_down': 'LWMC (5)',
    'right_up': 'RUMC (6)',
    'right_down': 'RWMC (7)',
    'unknown': 'unknown'
})

# 데이터 시각화 (2D 산점도)
fig = px.scatter(pca_df, x='PC1', y='PC2', color='Section', title='PCA 2D 시각화',
                category_orders={'Section': ['IO (1)', 'LMO (2)', 'RMO (3)', 'LUMC (4)', 'LWMC (5)', 'RUMC (6)', 'RWMC (7)', 'unknown']})

# 그래프의 크기와 해상도 조절
fig.update_layout(width=800, height=600)

fig.update_layout(
    font=dict(
        size=20  # 글꼴 크기를 20으로 설정
    )
)

# x,y 축 숫자 레이블의 글꼴 크기 설정
fig.update_layout(scene=dict(
    xaxis=dict(
        title_font=dict(size=17),
        tickfont=dict(size=10),   # x 축 숫자 레이블의 글꼴 크기를 설정
    ),
    yaxis=dict(
        title_font=dict(size=17),
        tickfont=dict(size=10),   # y 축 숫자 레이블의 글꼴 크기를 설정
    )
))

fig.show()


## 검증 코드

아래는 클러스터링이 제대로 됬는지 지정한 폴더의 엑셀파일들을 넣어보며 정확도를 검증하는 코드입니다.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

result_show = ""

# 현재 스크립트의 경로 가져오기
current_folder = os.getcwd()

folder_path = os.path.join(current_folder, 'train')

# 엑셀 파일 목록 얻어오기
excel_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.xlsx') and (not file_name.startswith("~$"))]

# 정답률을 계산하기 위한 변수 초기화
total_rows = 0
correct_predictions = 0

# 클래스 레이블을 추출합니다.
class_labels = ["front", "left_side", "right_side", "left_up", "left_down", "right_up", "right_down", "unknown"]

# 초기화: 정확한 예측 수를 누적할 딕셔너리
correct_predictions_count = {label: 0 for label in class_labels}

# 초기화: confusion matrix
confusion_matrix_result = np.zeros((len(class_labels), len(class_labels)), dtype=float)  # 타입을 float로 변경

# 각 파일별로 예측 실행 및 정확도 계산 및 컨퓨전 매트릭스 구성
for excel_file in excel_files:
    excel_file_path = os.path.join(folder_path, excel_file)

    # 엑셀 파일 읽기
    data_df = pd.read_excel(excel_file_path, engine='openpyxl')

    for index, row in data_df.iloc[:].iterrows():
        value_xx = row["xx"] if "xx" in data_df.columns else row[0]
        value_xy = row["xy"] if "xy" in data_df.columns else row[1]
        value_xz = row["xz"] if "xz" in data_df.columns else row[2]
        value_yx = row["yx"] if "yx" in data_df.columns else row[3]
        value_yy = row["yy"] if "yy" in data_df.columns else row[4]
        value_yz = row["yz"] if "yz" in data_df.columns else row[5]
        value_zx = row["zx"] if "zx" in data_df.columns else row[6]
        value_zy = row["zy"] if "zy" in data_df.columns else row[7]
        value_zz = row["zz"] if "zz" in data_df.columns else row[8]

        # 새로운 입력값을 2D 배열로 변환
        new_input_data = np.array([value_xx, value_xy, value_xz, value_yx, value_yy, value_yz, value_zx, value_zy, value_zz])

        # PCA 변환
        new_input_3d = pca.transform(new_input_data.reshape(1, -1))  # 3D로 변환

        # 모델을 활용하여 예측 수행
        predicted_section = model.predict(new_input_3d)[0]

        true_section_from_row = row.iloc[-1]

        section_mapping = {
            1: "front",
            2: "left_side",
            3: "right_side",
            4: "left_up",
            5: "left_down",
            6: "right_up",
            7: "right_down",
        }

        true_section_from_row = section_mapping.get(true_section_from_row, "unknown")

        print(f"File: {excel_file}, Row {index + 2} - Predicted section: {predicted_section}, True section: {true_section_from_row}")

        # 정확한 예측 수를 누적
        if predicted_section == true_section_from_row:
            correct_predictions_count[predicted_section] += 1

        # 컨퓨전 매트릭스 업데이트
        i = class_labels.index(true_section_from_row)
        j = class_labels.index(predicted_section)
        confusion_matrix_result[i, j] += 1

        total_rows += 1

        result_show += f"\nFile: {excel_file}, Row {index + 2} - Predicted section: {predicted_section}, True section: {true_section_from_row}"

    else:
        print("No valid data to calculate accuracy.")

# 정확도 계산
total_correct = sum(correct_predictions_count.values())
accuracy = total_correct / total_rows
print("Accuracy:", accuracy)

In [ ]:
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')

# confusion matrix 시각화 (Seaborn을 사용하여 시각화)
plt.figure(figsize=(20, 10),dpi=300)
ax = sns.heatmap(confusion_matrix_result / confusion_matrix_result.sum(axis=1)[:, np.newaxis], annot=True, cmap='Blues', fmt='.2f',annot_kws={'size': 22})  # 비율로 변환, annot=True로 설정하여 셀에 숫자 표시, cmap은 색상 맵, fmt은 숫자 포맷 지정
plt.xlabel('예측값',fontsize=20)
plt.ylabel('실제값',fontsize=20)
plt.title("K-means(pca5) 혼동행렬",fontsize=22)

class_labels = ["IO", "LMO", "RMO", "LUMC", "LWMC", "RUMC", "RWMC", "unknown"]

# 셀과 셀 이름의 위치 맞추기
ax.set_xticklabels(class_labels,rotation=0,fontsize=20)
ax.set_yticklabels(class_labels,rotation=0,fontsize=20)

plt.show()